In [1]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv


--2022-03-07 00:34:17--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.85.115
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.85.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699,83M  4,96MB/s    in 2m 7s   

2022-03-07 00:36:24 (5,51 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .master("local[*]") \
  .appName("fhvhv_tripdata") \
  .getOrCreate()

spark.sparkContext.uiWebUrl


/home/cristiandugacicu/.local/lib/python2.7/site-packages/pyspark/context.py:227: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [3]:
import pandas as pd
df_tripdata_csv = pd.read_csv("fhvhv_tripdata_2021-02.csv", nrows=1000)
df_tripdata_csv.dtypes

# loads Spark schema from Panda schema as an example to edit:
spark.createDataFrame(df_tripdata_csv).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [15]:
# Q2


from pyspark.sql import types

df_tripdata_schema = types.StructType([
    types.StructField("hvfhs_license_num",types.StringType(),True),
    types.StructField("dispatching_base_num",types.StringType(),True),
    types.StructField("pickup_datetime",types.TimestampType(),True),
    types.StructField("dropoff_datetime",types.TimestampType(),True),
    types.StructField("PULocationID",types.IntegerType(),True),
    types.StructField("DOLocationID",types.IntegerType(),True),
    types.StructField("SR_Flag",types.DoubleType(),True)
])


df_tripdata = spark.read \
      .option("header", "true") \
      .schema(df_tripdata_schema) \
      .csv("fhvhv_tripdata_2021-02.csv")

df_tripdata.repartition(24).write.parquet("pq/")


http://192.168.2.69:4041


In [16]:
# Q3

df_tripdata.registerTempTable("fhvhv_tripdata")

spark \
  .sql("""
    SELECT date_trunc('day', pickup_datetime) AS day, 
           count(1)
    FROM fhvhv_tripdata 
    GROUP BY day;
    """) \
  .show()

+-------------------+--------+
|                day|count(1)|
+-------------------+--------+
|2021-02-22 00:00:00|  382811|
|2021-02-14 00:00:00|  460661|
|2021-02-27 00:00:00|  509383|
|2021-02-11 00:00:00|  428288|
|2021-02-12 00:00:00|  469162|
|2021-02-26 00:00:00|  500049|
|2021-02-25 00:00:00|  436556|
|2021-02-23 00:00:00|  392696|
|2021-02-20 00:00:00|  497072|
|2021-02-13 00:00:00|  509331|
|2021-02-18 00:00:00|  372878|
|2021-02-03 00:00:00|  421697|
|2021-02-10 00:00:00|  403476|
|2021-02-08 00:00:00|  398445|
|2021-02-09 00:00:00|  398476|
|2021-02-24 00:00:00|  412556|
|2021-02-19 00:00:00|  459887|
|2021-02-06 00:00:00|  470555|
|2021-02-15 00:00:00|  367170|
|2021-02-16 00:00:00|  362596|
+-------------------+--------+
only showing top 20 rows



In [23]:
# Q4

df_tripdata.registerTempTable("fhvhv_tripdata")

spark \
  .sql("""
    SELECT hvfhs_license_num, pickup_datetime, dropoff_datetime,
           (unix_timestamp(dropoff_datetime)-unix_timestamp(pickup_datetime))/(60) as diff_min
    FROM fhvhv_tripdata 
    ORDER BY diff_min DESC
    ;
    """) \
  .show()


+-------------------+-------------------+------------------+
|    pickup_datetime|   dropoff_datetime|          diff_min|
+-------------------+-------------------+------------------+
|2021-02-11 13:40:44|2021-02-12 10:39:44|            1259.0|
|2021-02-17 15:54:53|2021-02-18 07:48:34| 953.6833333333333|
|2021-02-20 12:08:15|2021-02-21 00:22:14| 733.9833333333333|
|2021-02-03 20:24:25|2021-02-04 07:41:58|            677.55|
|2021-02-19 23:17:44|2021-02-20 09:44:01| 626.2833333333333|
|2021-02-25 17:13:35|2021-02-26 02:57:05|             583.5|
|2021-02-20 01:36:13|2021-02-20 11:16:19|             580.1|
|2021-02-18 15:24:19|2021-02-19 01:01:11| 576.8666666666667|
|2021-02-18 01:31:20|2021-02-18 11:07:15| 575.9166666666666|
|2021-02-10 20:51:39|2021-02-11 06:21:08| 569.4833333333333|
|2021-02-10 01:56:17|2021-02-10 10:57:33| 541.2666666666667|
|2021-02-25 09:18:18|2021-02-25 18:18:57|            540.65|
|2021-02-21 19:59:13|2021-02-22 04:56:16|            537.05|
|2021-02-09 18:36:13|202

In [36]:
# Q5

df_tripdata.registerTempTable("fhvhv_tripdata")

# 3 stages
# spark \
#   .sql("""
#     SELECT dispatching_base_num, 
#            count(1) AS c
#     FROM fhvhv_tripdata 
#     GROUP BY dispatching_base_num
#     ORDER BY c DESC
#     LIMIT 1;  ---adds another stage
#     """) \
#   .show()

# 3 stages
spark \
  .sql("""
    SELECT MAX (c) 
    FROM (SELECT dispatching_base_num, count(1) AS c
          FROM fhvhv_tripdata 
          GROUP BY dispatching_base_num);
    """) \
  .show()




+-------+
| max(c)|
+-------+
|3233664|
+-------+



In [76]:
# Q6
from pyspark.sql import types

df_zone_schema = types.StructType([
    types.StructField("LocationID",types.IntegerType(),True),
    types.StructField("Borough",types.StringType(),True),
    types.StructField("Zone",types.StringType(),True),
    types.StructField("service_zone",types.StringType(),True)
])

df_zone = spark.read \
      .option("header", "true") \
      .schema(df_zone_schema) \
      .csv("taxi+_zone_lookup.csv") \
      .select("LocationID","Zone")

df_zone_PU = df_zone.withColumnRenamed("zone", "PUzone").withColumnRenamed("LocationID", "PULocationID_")
df_zone_DO = df_zone.withColumnRenamed("zone", "DOzone").withColumnRenamed("LocationID", "DOLocationID_")

df_joined = df_tripdata \
  .join(df_zone_PU, df_tripdata.PULocationID ==  df_zone_PU.PULocationID_, 'inner') \
  .join(df_zone_DO, df_tripdata.DOLocationID ==  df_zone_DO.DOLocationID_, 'inner') 

df_joined.registerTempTable("fhvhv_tripdata_joined")
  

# spark \
#   .sql("""
#     SELECT *
#     FROM fhvhv_tripdata_joined 
#     """) \
#   .show()


spark \
  .sql("""
    SELECT CONCAT(PUzone, "-", DOzone) as pair,
           count(1) as c 
    FROM fhvhv_tripdata_joined 
    GROUP BY pair
    ORDER BY c DESC;
    """) \
  .show(10, truncate = False)

# spark \
#   .sql("""
#     SELECT CONCAT(PULocationID, "-", DOLocationID) as pair,
#            count(1) as c 
#     FROM fhvhv_tripdata 
#     GROUP BY pair
#     ORDER BY c DESC;
#     """) \
#   .show()

+-----------------------------------------+-----+
|pair                                     |c    |
+-----------------------------------------+-----+
|East New York-East New York              |45041|
|Borough Park-Borough Park                |37329|
|Canarsie-Canarsie                        |28026|
|Crown Heights North-Crown Heights North  |25976|
|Bay Ridge-Bay Ridge                      |17934|
|Astoria-Astoria                          |14688|
|Jackson Heights-Jackson Heights          |14688|
|Central Harlem North-Central Harlem North|14481|
|Bushwick South-Bushwick South            |14424|
|Flatbush/Ditmas Park-Flatbush/Ditmas Park|13976|
+-----------------------------------------+-----+
only showing top 10 rows



In [46]:
!cat taxi+_zone_lookup.csv